In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("key/bluebookkey.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': "https://bluebookai2020.firebaseio.com"
})


In [3]:
from firebase_admin import db

In [4]:
# retrive dataset from firebase
ref = db.reference("sat/raw/")
df_sat = ref.get()
df_sat=pd.DataFrame(df_sat).T.reset_index(drop="index")
df_sat.head(1)

,1_01,1_02,1_03,1_04,1_05,1_06,1_07,1_08,1_09,1_10,...,4_34,4_35,4_36,4_37,4_38,first,last,testDate,test_num,userID
0,C,B,D,A,C,D,D,A,A,D,...,3,4,2.6,3.,8,Test,Level,07/26/2019,R1,9999


In [5]:
ref = db.reference("sat/raw_ans/")
df_sat_ans = ref.get()
df_sat_ans=pd.DataFrame(df_sat_ans).T.reset_index(drop="index")
df_sat_ans.head(1)

,1_01,1_02,1_03,1_04,1_05,1_06,1_07,1_08,1_09,1_10,...,4_30,4_31,4_32,4_33,4_34,4_35,4_36,4_37,4_38,test_num
0,A,C,A,D,D,B,A,C,A,A,...,C,0.8,8,0.96,4.3;4.4;4.46;4.5;4.6;4.7,0.84,12,371,25.0,1


In [6]:
ref = db.reference("sat/raw_total_score/")
df_sat_score = ref.get()
df_sat_score=pd.DataFrame(df_sat_score).T.reset_index(drop="index")
df_sat_score.head(1)

,Math,Reading_Writing,first,last,testDate,test_num,total,userID
0,570.0,590,Shelly,Yi,4/2/18,2,1160,104


In [7]:
## making merge dataset
sat_score_col=['test_num',"testDate", "userID", "total"]
df_sat_score=df_sat_score[sat_score_col]

df_sat_score["testDate"]=pd.to_datetime(df_sat_score["testDate"])
df_sat["testDate"]=pd.to_datetime(df_sat["testDate"])

In [8]:
print(df_sat_score.shape, df_sat.shape)
df_merge=pd.merge(df_sat_score, df_sat, on=['userID',"testDate","test_num"])
df_merge=df_merge[~(df_merge.duplicated(["userID","testDate","test_num"], keep='first'))]
print(df_merge.shape)

(1144, 4) (853, 159)
(705, 160)


In [9]:
## nan count !! consider rows only 5 not answered 
row_index=[]
for row in range(df_sat.shape[0]):
    tmp=df_sat.iloc[row,:]
    if tmp.str.contains("nan").value_counts()[0] > 154:
        row_index.append(row)
len(row_index)

624

In [10]:
# new df_sat_not_null
df_sat_not_null=df_sat.iloc[row_index]

In [11]:
def ready_for_grade(df_sat, df_sat_ans, test_num):
    test_num = str(test_num)
    num_index=df_sat_ans.loc[df_sat_ans["test_num"]==f'{test_num}'].index[0]
    ans=df_sat_ans.iloc[num_index]
    
    # create questions by test_num
    df_sat_ans.iloc[:,-4:]
    userID=df_sat.loc[df_sat['test_num'] == f"{test_num}", "userID"]
    testDate=df_sat.loc[df_sat['test_num'] == f"{test_num}", "testDate"]
    tmp=df_sat.loc[df_sat['test_num'] == f"{test_num}"]
    data=tmp.iloc[:,:-5]
    
    
    
    return data, ans, userID, testDate

In [12]:
testNum = "9"
data, data_ans, userID, testDate=ready_for_grade(df_sat_not_null, df_sat_ans, testNum)

In [13]:
## final version

def grader_boolean(data, data_ans, userID, testDate):
    ans=data_ans
   # data=data
    #data.columns = userID
    
    for row in range(data.shape[0]):
        for col in range(len(data.columns)):
            tmp=pd.Series(ans[col]).str.contains(data.iloc[row,col])[0]
            if tmp:
                data.iloc[row,col]=True
            else:
                data.iloc[row,col]=False
                
    data["userID"] = userID
    data['testDate'] = testDate
    
    return data

In [14]:
df_graded = grader_boolean(data, data_ans, userID, testDate)

In [15]:
df_graded.head()

,1_01,1_02,1_03,1_04,1_05,1_06,1_07,1_08,1_09,1_10,...,4_31,4_32,4_33,4_34,4_35,4_36,4_37,4_38,userID,testDate
706,False,True,False,True,False,False,True,True,False,True,...,True,False,False,True,False,False,False,False,269,2016-11-09
707,True,False,False,True,False,True,True,True,True,True,...,True,False,False,False,False,True,False,False,23,2017-03-20
709,True,False,False,True,False,False,True,True,False,True,...,True,False,False,True,True,False,False,False,9001,2017-03-25
710,False,True,False,True,True,True,True,True,False,True,...,True,False,True,True,False,False,False,False,9968,2017-04-10
711,True,False,True,False,True,True,True,False,False,False,...,False,False,True,False,False,False,False,False,9798,2017-05-20


In [16]:
sat_col=['test_num','testDate', 'userID','total']
test_score=df_sat_score.loc[df_sat_score["test_num"] ==f"{testNum}",sat_col]

test_graded = df_graded



In [17]:
test_score["testDate"]=pd.to_datetime(test_score["testDate"])
test_graded["testDate"]=pd.to_datetime(test_graded["testDate"])


In [18]:
print(test_score.shape)
test_score.head(2)

(60, 4)


,test_num,testDate,userID,total
148,9,2017-03-20,23,1220
149,9,2017-12-29,46,1030


In [19]:
print(test_graded.shape)
test_graded.head(2)

(13, 156)


,1_01,1_02,1_03,1_04,1_05,1_06,1_07,1_08,1_09,1_10,...,4_31,4_32,4_33,4_34,4_35,4_36,4_37,4_38,userID,testDate
706,False,True,False,True,False,False,True,True,False,True,...,True,False,False,True,False,False,False,False,269,2016-11-09
707,True,False,False,True,False,True,True,True,True,True,...,True,False,False,False,False,True,False,False,23,2017-03-20


In [20]:
test_merge=pd.merge(test_score, test_graded, on=['userID',"testDate"])
print(test_merge.shape)
test_merge.head()

(13, 158)


,test_num,testDate,userID,total,1_01,1_02,1_03,1_04,1_05,1_06,...,4_29,4_30,4_31,4_32,4_33,4_34,4_35,4_36,4_37,4_38
0,9,2017-03-20,23,1220,True,False,False,True,False,True,...,True,True,True,False,False,False,False,True,False,False
1,9,2016-11-09,269,830,False,True,False,True,False,False,...,False,False,True,False,False,True,False,False,False,False
2,9,2017-12-21,358,1180,True,True,True,True,False,False,...,False,False,True,False,False,True,False,True,False,False
3,9,2017-09-01,417,1220,True,False,True,False,True,True,...,False,True,True,False,True,True,False,False,False,False
4,9,2017-08-18,433,930,True,True,True,False,False,True,...,False,False,True,False,False,False,False,False,False,False


In [21]:
sat_ml = test_merge.copy()
print(sat_ml.shape)
sat_ml['total']=sat_ml["total"].astype(int)

(13, 158)


In [22]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 20,
                              n_jobs = -1,
                              random_state = 1)
model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [23]:
def rmsle(y, y_, convertExp = False):
    if convertExp:
        y = np.exp(y),
        y_ = np.exp(y_)
    
    log1 = np.nan_to_num(np.array([np.log(v+1) for v in y])) 
    log2 = np.nan_to_num(np.array([np.log(v+1) for v in y_]))
    cal = (log1 - log2)**2
    return np.sqrt(np.mean(cal))
# np.nan_to_num when we want to replace nan(Not A Number) with zero and inf with finite numbers in an array  

def rmse(y, y_,):

    y = np.nan_to_num(np.array([v+1 for v in y])) 
    y_ = np.nan_to_num(np.array([v+1 for v in y_]))
    cal = (y - y_)**2
    return np.sqrt(np.mean(cal))


In [24]:
from tqdm import tqdm, trange
import numpy as np
import warnings
warnings.filterwarnings('ignore')


from sklearn.model_selection import train_test_split
n_loop = 200
col_list = []
for n in tqdm(range(n_loop)):
    max_depth=np.random.randint(100, 300)
    n_estimators=np.random.randint(150, 400)
    max_leaf_nodes=np.random.randint(2, 380)
    
    sample_size = np.random.randint(12,25)
    col_used=np.random.randint(5,158, size=sample_size)
    #X_train = df_sat
    X_train=sat_ml.iloc[:,col_used]
    X_train=X_train.astype(bool)
    #Y_train
    Y_train=pd.DataFrame(sat_ml["total"])
    
    X_train_cut, X_test_cut, y_train_cut, y_test_cut = \
        train_test_split(X_train, Y_train, test_size = 0.3, random_state = 1)

    model = RandomForestRegressor(n_estimators = n_estimators,
                                  max_depth = max_depth,
                                  max_leaf_nodes=max_leaf_nodes,
                              n_jobs = -1,
                              random_state = 1)
    
    %time model.fit(X_train_cut, y_train_cut)
    
    y_predict_test = model.predict(X_test_cut)
    rmsle_score=rmsle(y_predict_test, y_test_cut["total"])
    rmse_score = rmse(y_predict_test, y_test_cut["total"])
    sd_rmse = rmse(y_predict_test, y_test_cut["total"])/sat_ml["total"].std()
    
    
    col_list.append({"col_used": col_used, "sample_size": sample_size,"rmsle": rmsle_score, 
                     "rmse": rmse_score, "sd_rmse" : sd_rmse, "n_estimators": n_estimators,
                    "max_depth": max_depth, "max_leaf_nodes": max_leaf_nodes}) 
    print(pd.DataFrame(col_list).sort_values(by = "rmse").head(1))


  0%|          | 1/200 [00:00<01:15,  2.63it/s]

CPU times: user 222 ms, sys: 32.4 ms, total: 254 ms
Wall time: 253 ms
                                            col_used  max_depth  \
0  [107, 97, 12, 116, 123, 47, 123, 86, 146, 139,...        218   

   max_leaf_nodes  n_estimators        rmse     rmsle  sample_size   sd_rmse  
0             233           357  118.002803  0.108398           17  0.705719  


  1%|          | 2/200 [00:00<01:12,  2.74it/s]

CPU times: user 164 ms, sys: 21.3 ms, total: 186 ms
Wall time: 204 ms
                                            col_used  max_depth  \
1  [16, 56, 68, 7, 9, 79, 129, 68, 19, 77, 74, 93...        111   

   max_leaf_nodes  n_estimators        rmse     rmsle  sample_size   sd_rmse  
1             120           251  117.089143  0.111883           20  0.700255  


  2%|▏         | 3/200 [00:01<01:08,  2.88it/s]

...

CPU times: user 222 ms, sys: 29.2 ms, total: 251 ms
Wall time: 252 ms
                                              col_used  max_depth  \
131  [98, 48, 156, 140, 35, 58, 118, 16, 139, 146, ...        266   

     max_leaf_nodes  n_estimators       rmse     rmsle  sample_size   sd_rmse  
131              14           318  28.500348  0.029634           23  0.170447  


100%|█████████▉| 199/200 [01:10<00:00,  2.24it/s]

CPU times: user 265 ms, sys: 31.3 ms, total: 296 ms
Wall time: 321 ms
                                              col_used  max_depth  \
131  [98, 48, 156, 140, 35, 58, 118, 16, 139, 146, ...        266   

     max_leaf_nodes  n_estimators       rmse     rmsle  sample_size   sd_rmse  
131              14           318  28.500348  0.029634           23  0.170447  


100%|██████████| 200/200 [01:10<00:00,  2.32it/s]

CPU times: user 251 ms, sys: 35.3 ms, total: 286 ms
Wall time: 269 ms
                                              col_used  max_depth  \
131  [98, 48, 156, 140, 35, 58, 118, 16, 139, 146, ...        266   

     max_leaf_nodes  n_estimators       rmse     rmsle  sample_size   sd_rmse  
131              14           318  28.500348  0.029634           23  0.170447  


In [25]:
best_result=pd.DataFrame(col_list).sort_values(by = "rmsle").iloc[0]

In [26]:
best_result["col_used"]

array([ 98,  48, 156, 140,  35,  58, 118,  16, 139, 146, 126,  39,  64,
       143,  14,  73,  78, 113,  12,  35,  99, 129,  34])

In [27]:
best_qt=pd.DataFrame(col_list).sort_values(by = "rmsle").iloc[0]["col_used"]

In [28]:
model = RandomForestRegressor(n_estimators = best_result["n_estimators"],
                                  max_depth = best_result["max_depth"],
                                  max_leaf_nodes=best_result["max_leaf_nodes"],
                              n_jobs = -1,
                              random_state = 1)

In [29]:
X_train=sat_ml.iloc[:,best_qt]
X_train=X_train.astype(bool)

Y_train=pd.DataFrame(sat_ml["total"])

X_train_cut, X_test_cut, y_train_cut, y_test_cut = \
    train_test_split(X_train, Y_train, test_size = 0.3, random_state = 1)

%time model.fit(X_train_cut, y_train_cut)

y_predict_test = model.predict(X_test_cut)
rmsle_score=rmsle(y_predict_test, y_test_cut["total"])
rmse_score = rmse(y_predict_test, y_test_cut["total"])
sd_rmse = rmse(y_predict_test, y_test_cut["total"])/sat_ml["total"].std()

CPU times: user 198 ms, sys: 26.7 ms, total: 225 ms
Wall time: 238 ms


In [30]:
rmsle_score

0.029634090458798622

In [31]:
import math
math.sqrt(10)

df_predicted=pd.DataFrame(zip(y_predict_test, y_test_cut.values))
df_predicted.columns = ["p_score", 'a_score']
df_predicted['a_score']=df_predicted['a_score'].astype(int)
df_predicted['diff'] = np.abs(df_predicted['p_score'] - df_predicted['a_score'])

diff_mean=df_predicted.mean()[2]
diff_sd=df_predicted.std()[2]
confi_range= diff_mean + 1.9*diff_sd

print(diff_mean, confi_range, diff_sd)
df_predicted.head()


17.28773584905656 66.99874783827921 26.163690520643502


,p_score,a_score,diff
0,1181.729560,1180,1.729560
1,1213.301887,1220,6.698113
2,986.415094,930,56.415094
3,984.308176,980,4.308176


In [32]:
df_predicted.max()

p_score    1213.301887
a_score    1220.000000
diff         56.415094
dtype: float64

In [33]:
best_result=pd.DataFrame(best_result).T

In [34]:
best_result["diff_mean"] = diff_mean
best_result["diff_sd"] =diff_sd
best_result["confi_range"] = confi_range
best_result['diff_max'] = df_predicted.max()[2]


In [42]:
# test result
best_result.to_csv(f"test_result/best_result_test{testNum}.csv")


In [43]:
best_result

,col_used,max_depth,max_leaf_nodes,n_estimators,rmse,rmsle,sample_size,sd_rmse,diff_mean,diff_sd,confi_range,diff_max
131,"[98, 48, 156, 140, 35, 58, 118, 16, 139, 146, ...",266,14,318,28.5003,0.0296341,23,0.170447,17.287736,26.163691,66.998748,56.415094


In [37]:
#test1_merge.reset_index(drop="index").to_csv("test1_merge.csv")

In [38]:
"""
testR2_info=pd.read_csv("test_info_testNum_r2.csv").drop('Unnamed: 0', axis=1)
testR2_ans=pd.read_csv("test_answer_testNum_r2.csv").drop('Unnamed: 0', axis=1)

testR2_sat=pd.read_csv("r2/test_score_by_test_id_report.csv")
sat_col=['Test Date', 'Student ID','Total']
testR2_sat=testR2_sat[sat_col]
testR2_sat.columns= ["testDate", "userID", "total"]

testR2_sat["testDate"]=pd.to_datetime(testR2_sat["testDate"])
testR2_info["testDate"]=pd.to_datetime(testR2_info["testDate"])

testR2_merge=pd.merge(testR2_sat, testR2_info, on=['userID',"testDate"])
testR2_merge=testR2_merge[~(testR2_merge.duplicated(["userID","testDate"], keep='first'))]

testR2_merge.dropna()
"""

'\ntestR2_info=pd.read_csv("test_info_testNum_r2.csv").drop(\'Unnamed: 0\', axis=1)\ntestR2_ans=pd.read_csv("test_answer_testNum_r2.csv").drop(\'Unnamed: 0\', axis=1)\n\ntestR2_sat=pd.read_csv("r2/test_score_by_test_id_report.csv")\nsat_col=[\'Test Date\', \'Student ID\',\'Total\']\ntestR2_sat=testR2_sat[sat_col]\ntestR2_sat.columns= ["testDate", "userID", "total"]\n\ntestR2_sat["testDate"]=pd.to_datetime(testR2_sat["testDate"])\ntestR2_info["testDate"]=pd.to_datetime(testR2_info["testDate"])\n\ntestR2_merge=pd.merge(testR2_sat, testR2_info, on=[\'userID\',"testDate"])\ntestR2_merge=testR2_merge[~(testR2_merge.duplicated(["userID","testDate"], keep=\'first\'))]\n\ntestR2_merge.dropna()\n'

In [39]:
### coverting code for mergging 
"""
test1_info=pd.read_csv("test_info_testNum_1.csv").drop('Unnamed: 0', axis=1)
test1_ans=pd.read_csv("test_answer_testNum_1.csv").drop('Unnamed: 0', axis=1)

test1_sat=pd.read_csv("1/test_score_by_test_id_report_1.csv")

sat_col=['test_num','Test Date', 'Student ID','Total']
test1_sat=test1_sat[sat_col]
test1_sat.columns= ['test_num',"testDate", "userID", "total"]

test1_sat["testDate"]=pd.to_datetime(test1_sat["testDate"])
test1_info["testDate"]=pd.to_datetime(test1_info["testDate"])

test1_merge=pd.merge(test1_sat, test1_info, on=['userID',"testDate"])
test1_merge=test1_merge[~(test1_merge.duplicated(["userID","testDate"], keep='first'))]

test1_merge.head()
"""

'\ntest1_info=pd.read_csv("test_info_testNum_1.csv").drop(\'Unnamed: 0\', axis=1)\ntest1_ans=pd.read_csv("test_answer_testNum_1.csv").drop(\'Unnamed: 0\', axis=1)\n\ntest1_sat=pd.read_csv("1/test_score_by_test_id_report_1.csv")\n\nsat_col=[\'test_num\',\'Test Date\', \'Student ID\',\'Total\']\ntest1_sat=test1_sat[sat_col]\ntest1_sat.columns= [\'test_num\',"testDate", "userID", "total"]\n\ntest1_sat["testDate"]=pd.to_datetime(test1_sat["testDate"])\ntest1_info["testDate"]=pd.to_datetime(test1_info["testDate"])\n\ntest1_merge=pd.merge(test1_sat, test1_info, on=[\'userID\',"testDate"])\ntest1_merge=test1_merge[~(test1_merge.duplicated(["userID","testDate"], keep=\'first\'))]\n\ntest1_merge.head()\n'

In [40]:
"""
# collect data only less than 15 null values 
all_row=test1_merge.index
tmp = []
for row in all_row:
    tmp.append({"row": row,"null_count": test1_merge.shape[1]-pd.isnull(test1_merge.loc[row]).value_counts()[0]})
tmp=pd.DataFrame(tmp)
few_null_row=tmp.loc[tmp["null_count"] <5, "row"].values
"""

'\n# collect data only less than 15 null values \nall_row=test1_merge.index\ntmp = []\nfor row in all_row:\n    tmp.append({"row": row,"null_count": test1_merge.shape[1]-pd.isnull(test1_merge.loc[row]).value_counts()[0]})\ntmp=pd.DataFrame(tmp)\nfew_null_row=tmp.loc[tmp["null_count"] <5, "row"].values\n'

In [41]:
"""
def ml_ready(data,sat_boolean):
    
    sat_boolean.columns = data.columns[6:]
    sat_boolean=sat_boolean.reset_index()
    sat_boolean["userID"] = sat_boolean["index"]
    sat_boolean.drop("index",axis=1)
    sat_boolean=sat_boolean[list(["userID"]) + list(sat_boolean.columns[:-1])]
    data=data[["testDate","userID","total"]]
    
    # userID type from data = str = from sat_boolean
    data["userID"]=data["userID"].astype(str)
    
    # innder join
    sat_ml=data.merge(sat_boolean, left_on='userID', right_on='userID')
    
    return sat_ml
"""

'\ndef ml_ready(data,sat_boolean):\n    \n    sat_boolean.columns = data.columns[6:]\n    sat_boolean=sat_boolean.reset_index()\n    sat_boolean["userID"] = sat_boolean["index"]\n    sat_boolean.drop("index",axis=1)\n    sat_boolean=sat_boolean[list(["userID"]) + list(sat_boolean.columns[:-1])]\n    data=data[["testDate","userID","total"]]\n    \n    # userID type from data = str = from sat_boolean\n    data["userID"]=data["userID"].astype(str)\n    \n    # innder join\n    sat_ml=data.merge(sat_boolean, left_on=\'userID\', right_on=\'userID\')\n    \n    return sat_ml\n'